# Condor Game

The goal is to anticipate how asset prices will evolve by providing not a single forecasted value, but a **full probability distribution over the future price change at multiple forecast horizons and steps.**

## Probabilistic Forecasting

Probabilistic forecasting provides **a distribution of possible future values** rather than a single point estimate, allowing for uncertainty quantification. Instead of predicting only the most likely outcome, it estimates a range of potential outcomes along with their probabilities by outputting a **probability distribution**.

A probabilistic forecast models the conditional probability distribution of a future value $(Y_t)$ given past observations $(\mathcal{H}_{t-1})$. This can be expressed as:  

$$P(Y_t \mid \mathcal{H}_{t-1})$$

where $(\mathcal{H}_{t-1})$ represents the historical data up to time $(t-1)$. Instead of a single prediction $(\hat{Y}_t)$, the model estimates a full probability distribution $(f(Y_t \mid \mathcal{H}_{t-1}))$, which can take different parametric forms, such as a Gaussian:

$$Y_t \mid \mathcal{H}_{t-1} \sim \mathcal{N}(\mu_t, \sigma_t^2)$$

where $(\mu_t)$ is the predicted mean and $(\sigma_t^2)$ represents the uncertainty in the forecast.

Probabilistic forecasting can be handled through various approaches, including **variance forecasters**, **quantile forecasters**, **interval forecasters** or **distribution forecasters**, each capturing uncertainty differently.

In this notebook, we try to forecast the target location by a gaussian density function (or a mixture), the model output follows the form:

```python
[
    {
        "step": (k + 1) * step,
        "prediction": {
              "density": {
                            "name": "normal",
                            "params": {"loc": y_mean, "scale": y_var}
                          },
              "weight": weight
              }, ...
    }
    for k in range(0, horizon // step)
]
```

A **mixture density**, such as the gaussion mixture $\sum_{i=1}^{K} w_i \mathcal{N}(Y_t | \mu_i, \sigma_i^2)$ allows for capturing multi-modal distributions and approximate more complex distributions.

![proba_forecast_v3](https://github.com/Tarandro/image_broad/blob/main/proba_forecast_v3.png?raw=true)


**Probabilistic Forecasting** is particularly valuable in supply chain management. Below are some interesting resources for a deeper understanding:  

- [Probabilistic Forecasting](https://www.lokad.com/probabilistic-forecasting-definition/) – Overview of probabilistic forecasting and its applications.  
- [Quantile Forecasting](https://www.lokad.com/quantile-regression-time-series-definition/) – Explanation of quantile-based forecasting methods.  
- **Evaluation Metrics:**  
  - [Continuous Ranked Probability Score (CRPS)](https://www.lokad.com/continuous-ranked-probability-score/)  
  - [Cross-Entropy](https://www.lokad.com/cross-entropy-definition/)  
  - [Pinball Loss](https://www.lokad.com/pinball-loss-function-definition/)

In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from datetime import datetime, timezone, timedelta

from condorgame.price_provider import shared_pricedb
from condorgame.tracker import TrackerBase
from condorgame.tracker_evaluator import TrackerEvaluator
from condorgame.examples.utils import load_test_prices_once, load_initial_price_histories_once, count_evaluations
from condorgame.debug.plots import plot_quarantine, plot_prices, plot_scores

## What You Must Predict

Trackers must predict the **probability distribution of price changes**, defined as:

$$
r_{t,k} = P_t - P_{t-k}
$$

For each defined step **$k$** (e.g., 5 minutes, 1 hour, …), your tracker must return a full **probability density function (PDF)** over the future price change **$r_{t,k}$**.

# Gaussian Step Tracker

A simple benchmark that predicts future price changes by assuming they follow a Gaussian (normal) distribution estimated from recent historical data. It models the relative price change over each prediction step.

### **Key Ideas**  

- Historical prices sampled at 5-minute (300s) resolution are converted into returns: $r_{t} = P_t - P_{t-1}$
- The tracker estimates:
    - Drift: mean historical return 𝜇
    - Volatility: standard deviation historical returns 𝜎
- For each future step 𝑘, it outputs a normal density:
$$r_{t,k} \sim \mathcal{N}\!\left(\frac{k}{300}\mu,\; \sqrt{\frac{k}{300}}\sigma\right)$$

> The coefficient $\frac{k}{300}$ represents the ratio of the forecast step length to the base 5-minute (300s) interval, scaling the 5-minute return distribution to the target prediction step.

Each density prediction must comply with the [density_pdf](https://github.com/microprediction/densitypdf/blob/main/densitypdf/__init__.py) specification.

In [2]:
class GaussianStepTracker(TrackerBase):
    """
    A benchmark tracker that models *future incremental returns* as Gaussian-distributed.

    For each forecast step, the tracker returns a normal distribution
    r_{t,step} ~ N(a · mu, √a · sigma) where:
        - mu    = mean historical return
        - sigma = std historical return
        - a = (step / 300) represents the ratio of the forecast step duration to the historical 5-minute return interval.

    This is not a price-distribution; it is a distribution over 
    incremental returns between consecutive steps.
    """
    def __init__(self):
        super().__init__()

    def predict(self, asset: str, horizon: int, step: int):

        # Retrieve recent historical prices sampled at 5-minute resolution
        pairs = self.prices.get_prices(asset, days=5, resolution=300)
        if not pairs:
            return []

        _, past_prices = zip(*pairs)

        if len(past_prices) < 3:
            return []

        # Compute historical incremental returns (price differences)
        returns = np.diff(past_prices)

        # Estimate drift (mean return) and volatility (std dev of returns)
        mu = float(np.mean(returns))
        sigma = float(np.std(returns))

        if sigma <= 0:
            return []

        num_segments = horizon // step

        # Produce one Gaussian for each future time step
        # The returned list must be compatible with the `density_pdf` library.
        distributions = []
        for k in range(1, num_segments + 1):
            distributions.append({
                "step": k * step,
                "type": "mixture",
                "components": [{
                    "density": {
                        "type": "builtin",             # Note: use 'builtin' instead of 'scipy' for speed
                        "name": "norm",  
                        "params": {
                            "loc": (step/300) * mu, 
                            "scale": np.sqrt(step/300) * sigma
                            }
                    },
                    "weight": 1
                }]
            })

        return distributions

## Configurations

In [3]:
##########
# For each asset and historical timestamp, generate density forecasts
# over a fixed 24-hour horizon at multiple temporal resolutions
# (5 minutes, 1 hour, 6 hours, and 24 hours) and evaluate them
# against actual outcomes.

assets = ["BTC", "SOL"]

# Prediction horizon = 24h (in seconds)
HORIZON = 86400
# Prediction step sizes (multi-resolution forecast grid)
# All forecasts span the same 24h horizon but differ in temporal granularity.
STEP_CONFIG = {
    "5min":   300,
    "1hour":  300*12,
    "6hour":  300*12*6,
    "24hour": 300*12*24
}
# How often we evaluate the tracker (in seconds)
INTERVAL = 3600

# Base directory where all evaluation results will be stored
base_dir_results = "results"
os.makedirs(base_dir_results, exist_ok=True)

# End timestamp for the test data
# evaluation_end: datetime = datetime.now(timezone.utc)
evaluation_end: datetime = datetime(2025, 11, 15, 00, 00, 00, tzinfo=timezone.utc)

# Number of days of test data to load
# Note: the last `horizon` seconds of the time series will not be scored
days = 5

# Number of days of historical data used as warm-up before evaluation.
# This history is used only to initialize the tracker and is not scored.
days_history = 30

## Data

In [4]:
## Load the last N days of price data (test period)
test_asset_prices = load_test_prices_once(
    assets, shared_pricedb, evaluation_end, days=days
)

## Provide the tracker with initial historical data (for the first tick):
## load prices from the last H days up to N days ago
initial_histories = load_initial_price_histories_once(
    assets, shared_pricedb, evaluation_end, days_history=days_history, days_offset=days
)

## Run live simulation on historic data

In [5]:
# Setup tracker + evaluator
tracker_evaluator = TrackerEvaluator(GaussianStepTracker())

for asset, history_price in test_asset_prices.items():

    # First tick: initialize historical data
    tracker_evaluator.tick({asset: initial_histories[asset]})

    prev_ts = 0
    predict_count = 0
    pbar = tqdm(desc=f"Evaluating {asset}", total=count_evaluations(history_price, HORIZON, INTERVAL), unit="eval")
    for ts, price in history_price:
        # Feed the new tick
        tracker_evaluator.tick({asset: [(ts, price)]})

        # Evaluate prediction every hour (ts is in second)
        if ts - prev_ts >= INTERVAL:
            prev_ts = ts
            predictions_evaluated = tracker_evaluator.predict(asset, HORIZON, STEP_CONFIG)

            # Quarantine mechanism:
            # - Predictions are not scored immediately. Each prediction is placed in a quarantine 
            #   until sufficient future price data (up to the full horizon ticks) becomes available.
            # - Predictions issued within the final `horizon` seconds of the
            #   time series cannot be scored, as future observations are unavailable.

            if predictions_evaluated:
                pbar.update(1)

            # Periodically display results
            if predictions_evaluated and predict_count % 100 == 0:
                pbar.write(
                    f"[{asset}] avg CRPS={tracker_evaluator.overall_crps_score_asset(asset):.4f} | "
                    f"recent={tracker_evaluator.recent_crps_score_asset(asset):.4f}"
                )
            predict_count += 1
    pbar.close()
    print()

tracker_name = tracker_evaluator.tracker.__class__.__name__
print(f"\nTracker {tracker_name}:"
      f"\nFinal average crps score: {tracker_evaluator.overall_crps_score():.4f}")

current_results_dir = tracker_evaluator.to_json(horizon=HORIZON, step_config=STEP_CONFIG,
                                                interval=INTERVAL, base_dir=base_dir_results)

# Plot scoring timeline
timestamped_scores = tracker_evaluator.scores
plot_scores(timestamped_scores)

Evaluating BTC:  80%|████████  | 77/96 [02:51<00:56,  2.98s/eval]

[BTC] avg CRPS=22.5815 | recent=22.5815


Evaluating BTC: 100%|██████████| 96/96 [03:39<00:00,  2.29s/eval]


Evaluating SOL:  80%|████████  | 77/96 [03:13<00:51,  2.71s/eval]

[SOL] avg CRPS=23.2818 | recent=23.2818


Evaluating SOL: 100%|██████████| 96/96 [04:03<00:00,  2.53s/eval]




Tracker GaussianStepTracker:
Final average crps score: 24.9444
[✔] Tracker results saved to results\2025-11-11T00-00-00_to_2025-11-14T23-00-00\GaussianStepTracker_h86400.json


In [6]:
## Density forecast over returns (for the last asset and last prediction)
plot_quarantine(asset, predictions_evaluated[0], name_step="5min", prices=tracker_evaluator.tracker.prices, mode="direct")

In [7]:
## Return forecast mapped into price space (for the last asset and last quarantine prediction)
print("Normalized CRPS score:", tracker_evaluator.scores[asset][-1][1])
plot_quarantine(asset, predictions_evaluated[0], name_step="5min", prices=tracker_evaluator.tracker.prices, mode="incremental", lookback_seconds=HORIZON/4)

Normalized CRPS score: 27.539209399229758


# Tracker Comparison

In [8]:
from condorgame.examples.utils import load_all_results, plot_tracker_comparison

In [9]:
df_all = load_all_results(current_results_dir, horizon=HORIZON)
df_all

[✔] Found 1 files:
   - GaussianStepTracker_h86400.json


,tracker,asset,ts,score,time
0,GaussianStepTracker,BTC,1762819200,19.865815,2025-11-11 00:00:00+00:00
1,GaussianStepTracker,BTC,1762822800,18.458558,2025-11-11 01:00:00+00:00
2,GaussianStepTracker,BTC,1762826400,18.892219,2025-11-11 02:00:00+00:00
3,GaussianStepTracker,BTC,1762830000,20.057841,2025-11-11 03:00:00+00:00
4,GaussianStepTracker,BTC,1762833600,20.325400,2025-11-11 04:00:00+00:00
...,...,...,...,...,...
187,GaussianStepTracker,SOL,1763146800,29.355327,2025-11-14 19:00:00+00:00
188,GaussianStepTracker,SOL,1763150400,28.380778,2025-11-14 20:00:00+00:00
189,GaussianStepTracker,SOL,1763154000,28.123206,2025-11-14 21:00:00+00:00
190,GaussianStepTracker,SOL,1763157600,27.947281,2025-11-14 22:00:00+00:00


In [10]:
# Tracker comparison all assets (A lower CRPS score reflects more accurate predictions)
plot_tracker_comparison(df_all)

In [11]:
plot_tracker_comparison(df_all, 'SOL')

In [12]:
plot_tracker_comparison(df_all, 'BTC')